# El modelo de resultados potenciales

````{epigraph}
"La prediccion es muy dificil, especialmente si se trata del futuro."

-- Niels Bohr (citado en la pelicula *Oppenheimer*)
````

````{epigraph}
"Yo hago lo que me da la gana."

-- Bad Bunny
````

Imagina que pudieras vivir en un **multiverso**, donde en cada universo paralelo tomas una decision diferente y observas las consecuencias. En un universo estudias economia, en otro medicina, en otro decides no ir a la universidad. Si pudieras comparar tu vida en cada uno de esos universos, podrias medir exactamente el efecto que tuvo cada decision.

Este concepto no es solo ciencia ficcion. En 1957, el fisico Hugh Everett propuso la interpretacion de los muchos mundos de la mecanica cuantica, sugiriendo que cada decision cuantica genera una bifurcacion del universo. Aunque en econometria no tenemos acceso a universos paralelos, el **modelo de resultados potenciales** nos permite pensar de forma rigurosa sobre lo que *hubiera pasado* si las cosas fueran diferentes.

A lo que *hubiera pasado* le llamamos **contrafactual**: el resultado que habriamos observado en un escenario alternativo. El gran reto de la inferencia causal es que nunca podemos observar el contrafactual directamente. Solo podemos observar lo que realmente sucedio. El modelo de resultados potenciales nos da un marco formal para pensar en este problema.

## ¿Como seria tu vida si hubieras entrado en Harvard?

Una de las preguntas mas importantes en economia de la educacion es: **¿vale la pena asistir a una universidad mas selectiva?** La intuicion nos dice que si, pero la evidencia no es tan clara.

Gary Becker, premio Nobel de Economia, desarrollo la teoria del **capital humano**, que modela la relacion entre educacion e ingresos. En su forma mas simple, los ingresos de un individuo se pueden expresar como:

$$w = w_0 e^{rS}$$

donde $w_0$ es el salario base, $r$ es la tasa de retorno a la educacion y $S$ son los anos de escolaridad. Esta ecuacion sugiere que cada ano adicional de educacion incrementa los ingresos en una proporcion $r$.

Sin embargo, Heckman, Lochner y Todd mostraron que los retornos a la educacion no son iguales para todos: **difieren segun la habilidad del individuo, su entorno familiar y la calidad de la educacion recibida**. No es lo mismo un ano de educacion en una escuela rural que en una universidad de elite.

Esto nos lleva a la pregunta central: **¿asistir a una universidad mas selectiva incrementa los ingresos?**

Dale y Krueger (2002) estudiaron esta pregunta con datos de estudiantes que fueron **aceptados en universidades selectivas pero decidieron asistir a universidades menos selectivas**. Su hallazgo fue sorprendente: para la mayoria de los estudiantes, **asistir a una universidad selectiva no tiene un impacto significativo en los ingresos futuros**.

Es como la escena de *Spider-Man: No Way Home*, donde vemos multiples versiones de Spider-Man. Cada uno tomo caminos diferentes, pero al final, sus habilidades intrinsecas son las que determinan quien son. De forma similar, lo que importa no es tanto a que universidad asistes, sino las habilidades y la motivacion que ya traes contigo.

Pero hay una **excepcion importante**: para estudiantes de **bajos ingresos y minorias**, asistir a una universidad mas selectiva **si tiene un efecto positivo significativo** en sus ingresos futuros. Para estos estudiantes, la universidad selectiva abre puertas que de otra forma estarian cerradas: redes de contactos, oportunidades de empleo y senalizacion en el mercado laboral.

### Para identificar el efecto, debemos ir mas alla de la diferencia de medias

Un **error de novatos** en la identificacion causal es pensar que podemos medir el efecto de un tratamiento simplemente comparando los promedios de los grupos tratado y de control:

$$E[Y_1] - E[Y_0]$$

donde $Y$ es la variable de resultado (por ejemplo, ingresos), y $D$ es la variable de tratamiento. Usamos la siguiente notacion:

- $D_i = 1$: el individuo $i$ recibio el tratamiento (por ejemplo, asistio a una universidad selectiva)
- $D_i = 0$: el individuo $i$ no recibio el tratamiento
- $Y_{1i}$: resultado potencial del individuo $i$ si recibe el tratamiento
- $Y_{0i}$: resultado potencial del individuo $i$ si no recibe el tratamiento

El problema es que los individuos que reciben el tratamiento pueden ser **sistematicamente diferentes** de los que no lo reciben. Veamos esto con una simulacion.

Supongamos que los ingresos de un individuo $i$ estan determinados por:

$$Y_i = 1000 + 250 \cdot D_i + 250 \cdot A_i + \varepsilon_i$$

donde $D_i$ indica si el individuo asistio a una universidad selectiva, $A_i$ es su habilidad (no observada) y $\varepsilon_i$ es un termino de error aleatorio. El efecto **verdadero** de asistir a la universidad selectiva es exactamente $250$.

In [ ]:
import pandas as pd
import numpy as np
import random

alumnos = {'id': list(range(180))}
df = pd.DataFrame(alumnos)

random.seed(42)
np.random.seed(42)

df['habilidad'] = [np.random.normal(0, 1) for _ in range(len(df))]

ruido_habilidad = np.random.normal(0, 0.2, len(df))
df['selectivas'] = (df['habilidad'] + ruido_habilidad > 0.5).astype(int)

df['ingresos'] = 1000 + df['selectivas'] * 250 + df['habilidad'] * 250 + np.random.normal(0, 280, len(df))

df.head()

Veamos los promedios de ingresos por grupo:

| Grupo | Ingreso promedio |
|-------|------------------|
| Universidad selectiva ($D=1$) | ~1,530 |
| Universidad no selectiva ($D=0$) | ~854 |
| **Diferencia** | **~676** |

Calculemos la diferencia de medias directamente:

In [ ]:
df[df['selectivas'] == 1]['ingresos'].mean() - df[df['selectivas'] == 0]['ingresos'].mean()

La diferencia de medias es aproximadamente **676.14**, que es mucho mayor que el efecto verdadero de **250**. ¿Por que? Porque los estudiantes que asisten a universidades selectivas tienen, en promedio, **mayor habilidad** que los que no asisten. La diferencia de medias esta capturando no solo el efecto de la universidad, sino tambien la diferencia en habilidades entre los dos grupos.

## Sesgo de seleccion

![Sesgo de seleccion](../../figures/sesgo_seleccion.png)

El **sesgo de seleccion** ocurre cuando los individuos que reciben el tratamiento son sistematicamente diferentes de los que no lo reciben, de maneras que tambien afectan el resultado. En nuestro ejemplo, la **habilidad** es una variable que:

1. Afecta la probabilidad de asistir a una universidad selectiva (los estudiantes mas habiles tienen mas probabilidad de ser aceptados)
2. Afecta directamente los ingresos (los estudiantes mas habiles tienden a ganar mas, independientemente de la universidad)

Cuando la seleccion al tratamiento no es aleatoria, la simple comparacion de medias **no nos da el efecto causal**. En cambio, nos da el efecto causal **mas** el sesgo de seleccion.

## El beneficio de entrar a una escuela mas selectiva

Para entender mejor el problema, consideremos un ejemplo con dos individuos:

- **Alicia**: tiene alta habilidad y asistio a una universidad selectiva ($D_A = 1$)
- **Bernardo**: tiene baja habilidad y no asistio a una universidad selectiva ($D_B = 0$)

Observamos que Alicia gana mas que Bernardo. Pero, ¿podemos concluir que la diferencia se debe a la universidad? **No**, porque Alicia y Bernardo son diferentes en muchas dimensiones ademas de la universidad a la que asistieron.

Para formalizar esto, usamos la notacion de **resultados potenciales**:

- $Y_{1i}$: lo que ganaria el individuo $i$ si asistiera a una universidad selectiva
- $Y_{0i}$: lo que ganaria el individuo $i$ si **no** asistiera a una universidad selectiva
- $D_i$: indicador de si el individuo $i$ asistio a una universidad selectiva

El ingreso observado de cada individuo se puede escribir como:

$$Y_i = Y_{0i} + D_i (Y_{1i} - Y_{0i})$$

Esta ecuacion nos dice que el ingreso observado es igual al ingreso sin tratamiento mas el efecto del tratamiento (si es que lo recibio). El termino $(Y_{1i} - Y_{0i})$ es el **efecto causal individual** del tratamiento.

El problema fundamental es que para cada individuo solo observamos **uno** de los dos resultados potenciales. Para Alicia observamos $Y_{1A}$ pero no $Y_{0A}$. Para Bernardo observamos $Y_{0B}$ pero no $Y_{1B}$. El resultado que no observamos es el **contrafactual**:

$$E[Y_{0i} | D_i = 1]$$

es decir, ¿cuanto habrian ganado los que asistieron a la universidad selectiva si **no** hubieran asistido? Este es el contrafactual que necesitamos pero que nunca podemos observar directamente.

## Descubriendo el sesgo de seleccion

Podemos descomponer la diferencia de medias observada para revelar el sesgo de seleccion. Partimos de:

$$E[Y_i | D_i = 1] - E[Y_i | D_i = 0]$$

Usando la ecuacion de resultados potenciales $Y_i = Y_{0i} + D_i(Y_{1i} - Y_{0i})$, podemos escribir:

$$
\begin{aligned}
E[Y_i | D_i = 1] - E[Y_i | D_i = 0] &= E[Y_{1i} | D_i = 1] - E[Y_{0i} | D_i = 0] \\
&= E[Y_{1i} | D_i = 1] - E[Y_{0i} | D_i = 1] + E[Y_{0i} | D_i = 1] - E[Y_{0i} | D_i = 0] \\
&= \underbrace{E[Y_{1i} - Y_{0i} | D_i = 1]}_{\text{ATT}} + \underbrace{E[Y_{0i} | D_i = 1] - E[Y_{0i} | D_i = 0]}_{\text{Sesgo de seleccion}}
\end{aligned}
$$

Donde:

- **ATT** (Average Treatment Effect on the Treated): es el efecto promedio del tratamiento **sobre los tratados**. Es lo que realmente queremos medir: ¿cuanto beneficio obtuvieron, en promedio, los que asistieron a la universidad selectiva?

- **Sesgo de seleccion**: es la diferencia en los resultados potenciales *sin tratamiento* entre los grupos tratado y de control. Nos dice que tan diferentes eran estos grupos **antes** del tratamiento.

La ecuacion clave es entonces:

$$\text{Diferencia de medias} = \text{ATT} + \text{Sesgo de seleccion}$$

### ¿Como eliminamos el sesgo de seleccion?

La forma mas directa de eliminar el sesgo de seleccion es mediante la **aleatorizacion**. Si asignamos el tratamiento de forma aleatoria, entonces:

$$E[Y_{0i} | D_i = 1] = E[Y_{0i} | D_i = 0]$$

es decir, los grupos tratado y de control son, en promedio, iguales en todas las caracteristicas (observadas y no observadas). El sesgo de seleccion desaparece y la diferencia de medias nos da directamente el efecto causal.

![Sesgo de muestreo](../../figures/sampling_bias.JPG)

Un ejemplo clasico de sesgo de seleccion en la vida real es el estudio de **Bertrand y Mullainathan (2004)**, quienes enviaron curricula ficticios a ofertas de empleo en Estados Unidos. Los curricula eran identicos en todo excepto por el nombre del candidato: algunos tenian nombres que sonaban tipicamente afroamericanos (como Lakisha o Jamal) y otros tenian nombres que sonaban tipicamente blancos (como Emily o Greg). Los resultados mostraron que los curricula con nombres blancos recibieron un **50% mas de llamadas** para entrevistas. Este es un experimento de campo (**audit study**) que elimina el sesgo de seleccion al asignar aleatoriamente el "tratamiento" (el nombre en el curriculum).

## El experimento aleatorio ideal

Los **ensayos controlados aleatorios** (RCTs, por sus siglas en ingles) son considerados el **estandar de oro** para la identificacion causal. La idea es simple: si asignamos el tratamiento de forma aleatoria, eliminamos el sesgo de seleccion.

Un ejemplo clasico es el estudio de **Miguel y Kremer (2003)** sobre los efectos de la desparasitacion en la asistencia escolar en Kenia. Los investigadores asignaron aleatoriamente escuelas a recibir tratamiento de desparasitacion y midieron el efecto en la asistencia de los estudiantes. Encontraron que la desparasitacion redujo significativamente el ausentismo escolar.

Los pasos para disenar un experimento aleatorio son:

1. **Definir la poblacion de interes**: ¿a quienes queremos estudiar?
2. **Asignar aleatoriamente el tratamiento**: usar un mecanismo aleatorio (como lanzar una moneda o un generador de numeros aleatorios) para decidir quien recibe el tratamiento
3. **Medir los resultados**: observar la variable de resultado para ambos grupos (tratado y control)
4. **Comparar los promedios**: la diferencia de medias entre el grupo tratado y el grupo de control nos da una estimacion insesgada del efecto causal promedio

## Experimento ideal y experimentos naturales

En muchos casos, no es posible realizar un experimento aleatorio por razones eticas, logisticas o economicas. Por ejemplo, no podemos asignar aleatoriamente a personas a asistir o no a Harvard.

Aqui es donde entra la estrategia de **Dale y Krueger**: en lugar de aleatorizar, buscaron un grupo de comparacion **natural**. Compararon a estudiantes que fueron aceptados en las mismas universidades selectivas pero que eligieron asistir a universidades diferentes. Al condicionar en el conjunto de universidades a las que fueron aceptados, controlaron por la habilidad no observada.

Este tipo de estrategias se conocen como **experimentos naturales**: situaciones en las que algun evento o politica genera una variacion en el tratamiento que es *como si* fuera aleatoria, aunque no fue disenada explicitamente como un experimento. Los experimentos naturales son una de las herramientas mas poderosas de la econometria moderna y seran un tema recurrente a lo largo de este libro.

Algunos ejemplos clasicos de experimentos naturales incluyen:

- Loterias de reclutamiento militar (para estudiar el efecto del servicio militar en ingresos)
- Cambios en leyes de edad minima para beber (para estudiar el efecto del alcohol en mortalidad)
- Distancia a la universidad mas cercana (para estudiar el efecto de la educacion en ingresos)

## Disena tu experimento ideal con Inteligencia Artificial

Una forma interesante de practicar el diseno de experimentos es usar herramientas de inteligencia artificial como ChatGPT para ayudarte a pensar en los detalles. Aqui tienes un ejercicio:

1. **Elige una pregunta causal** que te interese (por ejemplo: ¿el teletrabajo aumenta la productividad?)
2. **Piensa en el tratamiento y el resultado**: ¿cual es la variable de tratamiento? ¿cual es la variable de resultado?
3. **Identifica posibles fuentes de sesgo de seleccion**: ¿por que los grupos tratado y de control podrian ser diferentes?
4. **Disena tu experimento ideal**: ¿como asignarias aleatoriamente el tratamiento?
5. **Usa el siguiente prompt en ChatGPT** para refinar tu diseno:

```
Soy un estudiante de econometria y estoy aprendiendo sobre inferencia causal.
Quiero disenar un experimento aleatorio ideal para responder la siguiente
pregunta causal: [TU PREGUNTA AQUI].

Por favor ayudame a:
1. Definir claramente la variable de tratamiento y la variable de resultado
2. Identificar posibles fuentes de sesgo de seleccion
3. Disenar un protocolo de aleatorizacion
4. Anticipar posibles problemas (incumplimiento, desercion, efectos derrame)
5. Sugerir un tamano de muestra razonable

Explicamelo de forma clara y concisa, como si fuera un capitulo de un
libro de texto.
```

Este ejercicio te ayudara a internalizar los conceptos de este capitulo y a pensar como un econometrista.

## Resumen

En este capitulo aprendimos los conceptos fundamentales del **modelo de resultados potenciales**, que es la base de la inferencia causal moderna:

- Cada individuo tiene dos **resultados potenciales**: $Y_{1i}$ (con tratamiento) y $Y_{0i}$ (sin tratamiento)
- Solo podemos observar **uno** de los dos resultados; el otro es el **contrafactual**
- La diferencia de medias entre tratados y no tratados captura el **efecto causal** mas el **sesgo de seleccion**
- El sesgo de seleccion surge cuando el tratamiento no se asigna de forma aleatoria
- La **aleatorizacion** elimina el sesgo de seleccion al hacer que los grupos sean comparables
- Cuando la aleatorizacion no es posible, los **experimentos naturales** ofrecen una alternativa

## Ejercicios

1. **Contrafactuales en la vida cotidiana**: Piensa en una decision importante que hayas tomado en tu vida (por ejemplo, elegir una carrera, mudarte a otra ciudad, etc.). ¿Cual seria el contrafactual? ¿Por que es imposible observarlo directamente?

2. **Sesgo de seleccion**: Un estudio encuentra que las personas que hacen ejercicio regularmente tienen menores tasas de depresion. ¿Podemos concluir que el ejercicio reduce la depresion? Identifica posibles fuentes de sesgo de seleccion.

3. **Resultados potenciales**: Supongamos que queremos medir el efecto de un programa de tutoria en las calificaciones de los estudiantes. Define formalmente $Y_{1i}$, $Y_{0i}$ y $D_i$ para este problema.

4. **Descomposicion de la diferencia de medias**: Usando la notacion del capitulo, demuestra que la diferencia de medias entre tratados y no tratados se puede descomponer en ATT + sesgo de seleccion.

5. **Simulacion en Python**: Modifica el codigo de simulacion de este capitulo para un caso donde el sesgo de seleccion es **negativo** (es decir, los individuos con peores resultados potenciales son los que reciben el tratamiento). Pista: cambia la relacion entre habilidad y seleccion al tratamiento.

6. **Diseno experimental**: Disena un experimento aleatorio para medir el efecto de ofrecer desayunos gratuitos en las escuelas sobre el rendimiento academico. Describe los pasos y las posibles dificultades.

7. **Experimentos naturales**: Investiga un ejemplo de experimento natural en la literatura economica (distinto a los mencionados en el capitulo). Describe la pregunta causal, la fuente de variacion exogena y los resultados principales.

8. **Audit study**: Inspirandote en el estudio de Bertrand y Mullainathan, disena un audit study para medir discriminacion en el mercado laboral de tu pais. ¿Que caracteristicas variarias en los curricula? ¿Que resultados medirias?

9. **Critica**: Un amigo te dice: "las personas que van a universidades privadas ganan mas dinero, por lo tanto las universidades privadas son mejores". Usando los conceptos de este capitulo, explica por que esta conclusion puede ser erronea.

10. **Reflexion etica**: ¿Existen preguntas causales para las cuales seria **poco etico** realizar un experimento aleatorio? Da un ejemplo y discute como un econometrista podria abordar esa pregunta de forma alternativa.